In [57]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

m = gp.Model("m")
# m.params.NonConvex = 0
m.params.InfUnbdInfo = 1

from collections import defaultdict
P_ABC = defaultdict(int)

# P_ABC[(0,0,1)] = 1/3
# P_ABC[(0,1,0)] = 1/3
# P_ABC[(1,0,0)] = 1/3

P_ABC[(0,0,0)] = 1/4
P_ABC[(0,0,1)] = 1/4
P_ABC[(0,1,0)] = 1/4
P_ABC[(1,0,0)] = 1/4

cardA, cardB, cardC = 2, 2, 2
Q_shape = (cardA, cardB, cardC, cardA, cardB, cardC)
n_col = np.prod(Q_shape)
print("Number of columns: ", n_col)

Q_base_old = m.addMVar(shape=Q_shape, vtype=GRB.CONTINUOUS, name="prob(A1,B1,C1,A2,B2,C2)", lb=0, ub=1)
Q_base = np.eye(64).reshape(Q_shape+(n_col,))


#probability and symmetry constraints (1,6)
# for a1, b1, c1, a2, b2, c2 in np.ndindex(cardA, cardB, cardC, cardA, cardB, cardC):
#     # m.addConstr(Q_base[a1,b1,c1,a2,b2,c2] >= 0)
#     # m.addConstr(Q_base[a1,b1,c1,a2,b2,c2] <= 1)
#     m.addConstr(Q_base[a1,b1,c1,a2,b2,c2] == Q_base[a2,b2,c2,a1,b1,c1])

#expressible set constraints
def P_AB(a,b):
    return sum([P_ABC[(a, b, C)] for C in range(cardC)])
def Q_AB1_AB2(a1, b1, a2, b2): #expr1 (2)
    return sum([Q_base[a1,b1,c1,a2,b2,c2] for c1, c2 in np.ndindex(cardC, cardC)])
def P_BC(b,c):
    return sum([P_ABC[(A, b, c)] for A in range(cardA)])
def Q_BC1_BC2(b1, c1, b2, c2): #expr2 (3)
    return sum([Q_base[a1,b1,c1,a2,b2,c2] for a1, a2 in np.ndindex(cardA, cardA)])
def P_AC(a,c):
    return sum([P_ABC[(a, B, c)] for B in range(cardB)])
def Q_AC1_AC2(a1, c1, a2, c2): #expr3 (4)
    return sum([Q_base[a1,b1,c1,a2,b2,c2]for b1, b2 in np.ndindex(cardB, cardB)])
def P_A(a):
    return sum([P_ABC[(a, B, C)] for B in range(cardB) for C in range(cardC)])
def P_B(b):
    return sum([P_ABC[(A, b, C)] for A in range(cardA) for C in range(cardC)])
def P_C(c):
    return sum([P_ABC[(A, B, c)] for A in range(cardA) for B in range(cardB)])
def Q_A1C1B2(a1,c1,b2): #expr4 (5)
    return sum([Q_base[a1,b1,c1,a2,b2,c2] for a2,c2,b1 in np.ndindex(cardA,cardC,cardB)])
def Q_A2C2B1(a2,c2,b1): #expr4 (5)
    return sum([Q_base[a1,b1,c1,a2,b2,c2] for a1,c1,b2 in np.ndindex(cardA,cardC,cardB)])



b_vec = []
b_vec_symbolic = []
A_mat = []

for a1, b1, a2, b2 in np.ndindex(cardA, cardB, cardA, cardB):
    A_mat.append(Q_AB1_AB2(a1, b1, a2, b2))
    b_vec.append(P_AB(a1,b1)*P_AB(a2,b2))
    b_vec_symbolic.append("*".join(sorted([f"P_AB({a1},{b1})",f"P_AB({a2},{b2})"])))
    # m.addConstr(Q_AB1_AB2(a1, b1, a2, b2) == P_AB(a1,b1)*P_AB(a2,b2))
for b1, c1, b2, c2 in np.ndindex(cardB, cardC, cardB, cardC):
    A_mat.append(Q_BC1_BC2(b1, c1, b2, c2))
    b_vec.append(P_BC(b1,c1)*P_BC(b2,c2))
    b_vec_symbolic.append("*".join(sorted([f"P_BC({b1},{c1})",f"P_BC({b2},{c2})"])))
for a1, c1, a2, c2 in np.ndindex(cardB, cardC, cardB, cardC):
    A_mat.append(Q_AC1_AC2(a1, c1, a2, c2))
    b_vec.append(P_AC(a1,c2)*P_AC(a2,c1))
    b_vec_symbolic.append("*".join(sorted([f"P_AC({a1},{c2})",f"P_AC({a2},{c1})"])))

for a1,c1,b2 in np.ndindex(cardA, cardC, cardB):
    # m.addConstr(Q_A1C1B2(a1,c1,b2) == P_A(a1)*P_C(c1)*P_B(b2))
    A_mat.append(Q_A1C1B2(a1,c1,b2))
    b_vec.append(P_A(a1)*P_C(c1)*P_B(b2))
    b_vec_symbolic.append(f"P_A({a1})*P_C({c1})*P_B({b2})")
    A_mat.append(Q_A2C2B1(a1,c1,b2))
    b_vec.append(P_A(a1)*P_C(c1)*P_B(b2))
    b_vec_symbolic.append(f"P_A({a1})*P_C({c1})*P_B({b2})")

A = np.asarray(A_mat)
b = np.asarray(b_vec)
y = m.addMVar(shape=len(b), vtype=GRB.CONTINUOUS, name="y", lb=-1, ub=1)
# m.addConstr(y.sum() >= -1)
# m.addConstr(y.sum() == 1)
m.setObjective(y @ b, GRB.MINIMIZE)
m.addMConstr(A.T, y, '>=', [0]*n_col)
# m.addMConstr(A, Q_base_old.reshape(-1), '=', b)
m.update()


m.optimize()
# m.computeIIS()
# cert = m.getAttr('FarkasDual')
# from scipy.sparse import coo_matrix
# print(coo_matrix(cert))
# 
# for a1, b1, c1, a2, b2, c2 in np.ndindex(cardA, cardB, cardC, cardA, cardB, cardC):
#     print(f"Q({a1},{b1},{c1},{a2},{b2},{c2}) = {Q_base[a1,b1,c1,a2,b2,c2].X}")
print("Objective value: ", m.objVal)
from collections import defaultdict
solution_dict = defaultdict(int)
for name, val in zip(b_vec_symbolic, y.X):
    if val != 0:
        solution_dict[name] += val
for key, val in solution_dict.items():
    if val != 0:
        print(f"{key} * {val}")
# print("Optimal solution:", y.X)

Set parameter InfUnbdInfo to value 1
Number of columns:  64
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 64 rows, 128 columns and 320 nonzeros
Model fingerprint: 0xf6faba22
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 13 rows and 85 columns
Presolve time: 0.00s
Presolved: 51 rows, 43 columns, 210 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -3.8750000e+00   1.920000e+02   0.000000e+00      0s
      28    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Objective value:  0.0
P_AB(0,0)*P_AB